# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [4]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [5]:
# enter your code here
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

Sub Problem 1: How to aggregate the `amount_spent` for unique customers?


In [15]:
# your code here
unique_spending_cust = orders[['amount_spent']].groupby(orders['CustomerID']).sum()
unique_spending_cust.head()

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40


Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

In [58]:
quant_VIP = 0.95
quant_preferred = 0.75

def quant_orders(x):
    return unique_spending_cust.quantile(x)


In [59]:
VIP = quant_orders(quant_VIP)
VIP

amount_spent    5840.182
Name: 0.95, dtype: float64

In [60]:
preferred = quant_orders(quant_preferred)
preferred

amount_spent    1661.64
Name: 0.75, dtype: float64

 Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

In [74]:
VIPS = unique_spending_cust\
[unique_spending_cust['amount_spent'] > VIP[0]]

VIPS

,amount_spent
CustomerID,
12346,77183.60
12357,6207.67
12359,6372.58
12409,11072.67
12415,124914.53
...,...
18109,8052.97
18139,8438.34
18172,7561.68


In [73]:
pref = unique_spending_cust\
 [(unique_spending_cust['amount_spent'] > preferred[0])
& (unique_spending_cust['amount_spent']< VIP[0])]
pref

,amount_spent
CustomerID,
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
12356,2811.43
...,...
18259,2338.60
18260,2643.20
18272,3078.58


In [75]:
orders.set_index('CustomerID')

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,Country,amount_spent
CustomerID,,,,,,,,,,,,,
17850,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,United Kingdom,15.30
17850,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34
17850,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,United Kingdom,22.00
17850,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34
17850,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12680,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,France,10.20
12680,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,France,12.60
12680,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,France,16.60


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

Provide your solution for Q2 below:

In [88]:
orders.loc[VIPS.index].groupby(['Country']).count()['CustomerID']

Country
EIRE                3
France              6
Germany             6
Japan               1
Norway              6
Portugal            2
Spain               2
United Kingdom    191
Name: CustomerID, dtype: int64

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [90]:
# your code here this takes preferred and vip combined
pref_vip = unique_spending_cust[(unique_spending_cust['amount_spent'] > preferred[0])]
pref_vip

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12352,2506.04
...,...
18259,2338.60
18260,2643.20
18272,3078.58


In [91]:
orders.loc[pref_vip.index].groupby(['Country']).count()['CustomerID']

Country
Channel Islands      2
Denmark              8
EIRE                12
France              19
Germany             22
Japan                6
Norway              24
Portugal             6
Spain               15
United Kingdom     971
Name: CustomerID, dtype: int64